In [1]:
# Read in test data
filepath = "day22_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

In [2]:
from collections import defaultdict
import copy 
player_cards = defaultdict(list)
player = 1
for _ in lines:
    if _ == "":
        player += 1
    else:
        try:
            player_cards[player].append(int(_))
        except:
            pass

In [3]:
p1_cards = copy.deepcopy(player_cards[1])
p2_cards = copy.deepcopy(player_cards[2])

In [4]:
game_round = 0
def playGame(p1_cards, p2_cards):
    
    global game_round
    game_round += 1
    i = 0
    
    # update to a sit to clean up search
    memory_list_p1 = set()
    memory_list_p2 = set()
        
    while True:
        i += 1
        
        # check for previous round -> passed test case
        if tuple(p1_cards) in memory_list_p1 and tuple(p2_cards) in memory_list_p2:
            return (p1_cards, p2_cards, 1)
        
        # add snapshot of cards
        memory_list_p1.add(tuple(p1_cards))
        memory_list_p2.add(tuple(p2_cards))
        #memory_list.append(copy.deepcopy(p1_cards) + copy.deepcopy(p2_cards)) 
        

        # get top cards
        top_p1 = p1_cards.pop(0)
        top_p2 = p2_cards.pop(0)

        # check for recursive combat: 
        if top_p1 <= len(p1_cards) and top_p2 <= len(p2_cards):
            
            # we play game and output 
            p1_copy = p1_cards[:top_p1].copy()
            p2_copy = p2_cards[:top_p2].copy()
            trash1, trash2, winner = playGame(p1_copy, p2_copy)
            
            # check if we had a winner from prior game 
            if winner == 1:
                p1_cards.append(top_p1)
                p1_cards.append(top_p2) 

            if winner == 2:
                p2_cards.append(top_p2)
                p2_cards.append(top_p1) 

        else:
            # update based on top card
            if top_p1 > top_p2:
                p1_cards.append(top_p1)
                p1_cards.append(top_p2)
            else:
                p2_cards.append(top_p2)
                p2_cards.append(top_p1)

        # if either player hits 0, break
        if len(p1_cards) == 0 or len(p2_cards) == 0:
            if len(p1_cards) > 0:
                return (p1_cards, p2_cards, 1)
            else:
                return (p1_cards, p2_cards, 2)

In [5]:
p1_cards = copy.deepcopy(player_cards[1])
p2_cards = copy.deepcopy(player_cards[2])
p1_list, p2_list, winner = playGame(p1_cards, p2_cards)

In [6]:
tot = 0
for i, val in enumerate(p1_list[::-1]):
    tot += val * (i+1)
    
print(tot)

33206


### Issues I had to Resolve Later:

- Was able to get proper output for the test case, but knew something was slightly off given how long my code took to run AND given I had two answers, one which was too high and one was too low (difference of about 200 so no use in guessing!)

- Issue Resolved Around Storing Memory: 
    - Thought through it & realized I had been concatenating the p1 & p2 card set each round, which is an issue. 
    - If p1 had `1,4,5,6` and p2 had `2,7,8,9` then i stored this as: `1,4,5,6,2,7,8,9`
    - Problem: This would flag a repeat for the wrong hands:
        - p1 had `1,4,5,6,2,7` and p2 had `8,9` got incorrectly flagged
        - p2 had `1,4` and p2 had `5,6,2,7,8,9` got incorrectly flagged
    - Fix: 
        - Use sets to be faster in general (need to store list as tuple due to mutability of lists)
        - Use two separate types of storage for p1 vs p2 instead of concatenation. 
